In [1]:
from z3 import *

In [8]:
def rid1(x,y): # a receptive relations. It receives variables
    return x == y

def compose1(f, sort, g): # annoying but we need to supply the sort of the inner variable
    def res(x,z):
        y = FreshConst(sort)
        return Exists([y], And(f(x,y), g(y,z)))
    return res

def rid2(x): # a functional relation. It receives a variable then produces one.
    y = FreshConst(x.sort())
    return y, x == y

def compose2(f,g):
    def res(x):
       y, cf = f(x)
       z, cg = g(y)
       return z, Exists([y], And(cf,cg) )
    return res

def rid3(sort):  # a type indexed generator of relations. Annoying we have to supply the type of the variable
    def res(): # a productive relation
       x = FreshConst(sort)
       y = FreshConst(sort)
       return x, y, x == y
    return res

def compose3(f,g):
   def res():
      x, yf, cf = f()
      yg, z, cg = g()
      return x, z, Exists([yf,yg], And(cf, cg, yf == yg))
   return res

The different styles are interconvertible with each other. We need to supply the appropriate sort information unfortunately.

In [9]:
def lift12(sorty, f):
    def res(x):
        y = FreshConst(sorty)
        c = f(x,y)
        return y, c
    return res

def lift23(sortx, f):
    def res():
        x = FreshConst(sortx)
        y, c = f(x)
        return x, y, c
    return res
  
def lift31(f):
    def r(x,y):
      x1, y1, c = f()
      return x1, y1, And(c, x1 == x, y1 == y)
    return res

Relations are sets of tuples. Predicates, which are z3 expressions of sort `a -> Bool` are also reasonable representations of sets. This functional representation wasn't so great in Haskell, because we needed to introspect into the lambda functions in order to do many search-like operations. Z3 has no qualms doing this.

We can create the general cadre of relation algebra operators.



In [ ]:
trivial = BoolVal(True)

def top1(x,y): # top is the full relation,
    return trivial
def bottom1(x,y):
    return BoolVal(False)

def top2(sorty):
    def res(x):
        y = FreshConst(sorty)
        return y, trivial
    return res

def top3(sortx, sorty):
    def res():
        x = FreshConst(sortx)
        y = FreshConst(sorty)
        return x, y, trivial
    return res

def converse1(r):
    return lambda x,y: r(y,x)
def meet1(p,q):
    return lambda x,y : p(x,y) & q(x,y)
def join1(p,q):
    return lambda x,y : p(x,y) | q(x,y)

# product and sum types
def fst1(x,y): # proj(0)
    return y == x.sort().accessor(0,0)(x)
def snd1(x,y): # proj(1)
    return y == x.sort().accessor(0,1)(x)
def left1(x,y):
    return y == y.sort().constructor(0)(x)
def right1(x,y):
    return y == y.sort().constructor(1)(x)
def inj1(n):
    return lambda x, y: return y == y.sort().constructor(n)(x)
def proj1(n):
    return lambda x, y: return y == x.sort().accessor(0,n)(x)
def fan(p,q):
    def res(x,y): 
       a = y.sort().accessor(0,0)(y)
       b = y.sort().accessor(0,1)(y)
       return And(p(x,a), q(x,b))
    return res
def dup1(x,(y1,y2)): # alternatively we may not want to internalize the tuple into z3.
    return And(x == y1 , x == y2)
def convert_tuple((x,y), xy):  # convert between internal z3 tuple and python tuple.
    return xy == xy.constructor(0)(x,y)
#def split():
#def rdiv

In [ ]:
def lte1(x,y):
    return x <= y
def sum1(x,y): # I'm being hazy about what x is here exactly
    return x[0] + x[1] == y
def npsum(x,y):
    return np.sum(x) == y # you can make numpy arrays of z3 variables. Some vectorized functions work.
def mul1(x,y):
    return x[0] * x[1] == y
def and1((x,y), z):
    return z == And(x,y)
def If1((b,t,e),y):
    return If(b, t,e) == y

Relational properties can be expressed as a ForAll statement

In [ ]:
# relational properties

def rsub(p,q):
    return ForAll([x,y].Implies(p(x,y) , q(x,y) ))
def req(p,q):
    return And(rsub(p,q), rsub(q,p) 
def symmetric1(sortx, sorty, r):
    x = FreshConst(sortx)
    y = FreshConst(sorty)
    return ForAll([x,y], r(x,y) == r(y,x))
def reflexive1(sortx, r):
    x = FreshConst(sortx)
    return ForAll([x],r(x,x))
def transitive1(sortx,sorty,sortz, r):
    x = FreshConst(sx)
    y = FreshConst(sy)
    ForAll([x,y], Implies(r(x,y) & r(y,z) , r(x,z))
def strict1(r):
    return Not(r(x,x))
def itern(n, sortx, p): # unroll 
    if n == 0:
        return rid1(sortx)
    else:
        return compose(starn(n-1, sortx, p), p)
def starn(n, sortx, p): # unroll and join
    if n == 0:
        return rid1(sortx)
    else:
        return join(rid, compose(starn(n-1, sortx, p))) # 1 + x * p 
           


In [ ]:
# Abstract Relational Reasoning.
# Uninterpeted Sorts.

Z3 has solvers for

- Combinatorial Relations
- Linear Relations
- Polyhedral Relations
- Polynomial Relations
- Interval Relations - A point I was confused on. I thought interval relations were not interesting. But I was interpetting the term incorrectly. I was thinking of relations on AxB that are constrained to take the form of a product of intervals. In this case, the choice of A has no effect on the possible B whatsoever, so it feels non relational. However, there is also I_A x I_B , relations over the intervals of A and B. This is much closer to what is actually being discussed in interval arithmetic.  

What can we use this for:

- Graph Problems. The Edges can be thought of as a relation between vertices. Relation composition Using the `starn` operator is a way to ask for paths through the graph.
- Linear Relations - To some degree this might supplant my efforts on linear relations. Z3 is fully capable of understanding linear relations.
- Safety and liveness of control systems. Again,a transition relation is natural here. It is conceivable that the state space
- Program verification




Power Set and elem Relation

Higher order relations?

Galois connections and QR iteration
QS = SQ*... Na. I don't think this makes much sense. 

I think most proof assistants have implementations of relation algebra available. I find you can do a surprising amount in z3. 
Where z3 is weaker is in the presence of many alternating quantifiers. These are hard problems and they require more human guidance.



Thoughts
- Maybe we are just building a shitty alloy.
- Z3 has special support for some relations. How does that play in?  https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-special-relations https://z3prover.github.io/api/html/ml/Z3.Relation.html 
- What we've done here is basically analogous/identical to what John Wiegley did compiling to the category of z3. Sligthly different in that he only allowed for existential composition rather than relational division.
- We can reduced the burden on z3 if we know the constructive proof objects that witness our various operations. Z3 is gonna do better if we can tell it exactly which y witness the composition of operators, or clues to which branch of an Or it should use.
- It's a bummer, but when you use quantifiers, you don't see countermodels? Maybe there is some hook where you can, or in the dump of the proof object.
- What about recursion schemes? The exact nature of z3 to handle unbounded problems is fuzzy to me. It does have the support to define recursive functions. Also explicit induction predicates can go through sometimes. 
def fmap(p):    
def cata(p):
    # find self recursive calls using sort.
    ForAll([self], )
    ForAll([], cata(p) == compose(p , cata(p)) 